In [61]:
import sqlite3
from sqlite3 import Error
import os.path
from os import path

#from create_db import create_connection, create_table, clean_column_names
#from populate_db import extract_data, insert_records
#import query_db as qd
import config
import build_dataset as build
import pipeline
import classification

import importlib
import datetime
import re
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_score, PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix, classification_report, plot_roc_curve, plot_precision_recall_curve 

# Decision Tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
%matplotlib inline

In [62]:
importlib.reload(build)
importlib.reload(config)
importlib.reload(pipeline)
importlib.reload(classification)

<module 'classification' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/classification.py'>

# Notebook Table of Contents:
1. Get and fit the best model
3. Import active sentence data
2. Predict recidivism levels using active sentences and active (almost complete) sentences
4. Store results and graphs

In [63]:
# Import data - trimmed to 1994 here
dataset_3 = pd.read_csv('../data/dataset_main_active3.csv')

/Users/daminisharma/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
# Getting sum stats
target = config.target_vars[0]
features = "Demographics"
target_type = "all"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)


248
Size of active sentences dataset:  31360
Dataset size:  527096
Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent'],
      dtype='object')


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [72]:
total = dataset_no_active_norm.groupby(target).size().reset_index(name='Count')
total['Pct'] = total['Count']/dataset_no_active_norm.shape[0]
total

,Recidivate_Risk_Level,Count,Pct
0,0.0,366535,0.694683
1,1.0,37290,0.070675
2,2.0,43981,0.083356
3,3.0,61866,0.117253
4,4.0,15530,0.029434
5,5.0,2427,0.004600


In [68]:
# Getting sum stats
target = config.target_vars[0]
features = "Demographics"
target_type = "all"
year = 2008
train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)


248
Size of active sentences dataset:  27486
Dataset size:  224981
Total Number of Unique IDs: 164194
Total Number of IDs in Test Data: 32839
Total Number of IDs in Train Data: 105084
Total Number of IDs in Validate Data: 26271
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.2000012180713059
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.7999987819286941
Does Validate Represent 20% of the Train+Validate Data?: 0.2
Does Train Represent 80% of the Train+Validate Data?: 0.8


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent'],
      dtype='object')


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [70]:
total = dataset_no_active_norm.groupby(target).size().reset_index(name='Count')
total['Pct'] = total['Count']/dataset_no_active_norm.shape[0]
total

,Recidivate_Risk_Level,Count,Pct
0,0.0,171163,0.759182
1,1.0,12174,0.053997
2,2.0,11736,0.052054
3,3.0,25668,0.113849
4,4.0,3991,0.017702
5,5.0,725,0.003216


In [75]:
#

In [60]:
# Run train/test for 1994 binary demographics
# Run predict for 1994 binary demographics

classifier = 'DecisionTreeClassifier'
results_path = '../results/1994/best_models_binary_Demographics.csv'
target = config.target_vars[0]
features = "Demographics"
target_type = "binary"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 1994 three_class demographics
# Run predict for 1994 three_class demographics


target_type = "three_class"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 1994 six_class demographics
# Run predict for 1994 six_class demographics

target_type = "all"
year = 1994

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 2008 binary demographics
# Run predict for 2008 binary demographics

target_type = "binary"
year = 2008

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)

# Run train/test for  2008 three_class demographics
# Run predict for 2008 three_class demographics

target_type = "three_class"
year = 2008

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)


# Run train/test for 2008 six_class demographics
# Run predict for 2008 six_class demographics

target_type = "all"
year = 2008

train_data, test_data, validate_data, active_sentences, active_almost_complete, dataset_no_active_norm = build.split_and_process(dataset_3, config,target_type,features,year)
active_prob, almost_complete_prob = classification.predict_using_best(classifier, results_path, dataset_no_active_norm, active_sentences, active_almost_complete,target_type, year)





248
Size of active sentences dataset:  31360
Dataset size:  527096


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

Total Number of Unique IDs: 304786
Total Number of IDs in Test Data: 60957
Total Number of IDs in Train Data: 195063
Total Number of IDs in Validate Data: 48766
Do the IDs add up? True
Does Test Represent 20% of the data? False
Test Represents X% of the data: 0.19999934380188067
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8000006561981193
Does Validate Represent 20% of the Train+Validate Data?: 0.20000082024697635
Does Train Represent 80% of the Train+Validate Data?: 0.7999991797530236


/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')
Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sen

...

Index(['RACE', 'GENDER', 'ETHNICITY', 'CONTROL_STATUS', 'OFFENSE_CLEAN',
       'Current_Offense_Risk_Level', 'year_month', 'INFRACTION_PER_SENT',
       'misd_count', 'felon_count', 'sentence_count', 'age_at_sentence',
       'unemp_rate', 'ID', 'COMMITMENT_PREFIX', 'BIRTH_DATE',
       'EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE',
       'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'Recidivate_Risk_Level',
       'age_cat', 'age_first_offense', 'juv_first_offense',
       'current_crime_violent'],
      dtype='object')
Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sen

/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/build_dataset.py:693: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_first_offense.drop_duplicates(inplace=True)
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode().values[0])
/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/pipeline.py:190: SettingWithCo

Index(['Recidivate_Risk_Level', 'RACE', 'GENDER', 'ETHNICITY',
       'CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix', 'age_cat',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent'],
      dtype='object')


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Training... DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1234, splitter='best')
Predicting...
Number of active sentences for 2008 : 27486
Number of active sentences for 2008 : 9427
Likelihood of recidivism (all active sentences) for 2008 :
   group  count  pct
0    0.0  27486  1.0


Likelihood of recidivism (almost complete active sentences) for 2008 :
   group  count  pct
0    0.0   9427  1.0
